In [1]:
from __future__ import division, print_function
%matplotlib inline
from importlib import reload  # Python 3
import utils; reload(utils)
from utils import *

WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5110)
Using TensorFlow backend.


## Setup

We're going to download the collected works of Nietzsche to use as our data for this class.

In [2]:
path = get_file('nietzsche.txt', origin="https://s3.amazonaws.com/text-datasets/nietzsche.txt")
text = open(path).read()
print('corpus length:', len(text))

corpus length: 600893


In [3]:
chars = sorted(list(set(text)))
vocab_size = len(chars)+1
print('total chars:', vocab_size)

total chars: 85


Sometimes it's useful to have a zero value in the dataset, e.g. for padding

In [4]:
chars.insert(0, "\0")

In [5]:
''.join(chars[1:-6])

'\n !"\'(),-.0123456789:;=?ABCDEFGHIJKLMNOPQRSTUVWXYZ[]_abcdefghijklmnopqrstuvwxy'

Map from chars to indices and back again

In [6]:
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

*idx* will be the data we use from now own - it simply converts all the characters to their index (based on the mapping above)

In [7]:
idx = [char_indices[c] for c in text]

In [8]:
idx[:10]

[40, 42, 29, 30, 25, 27, 29, 1, 1, 1]

In [9]:
''.join(indices_char[i] for i in idx[:70])

'PREFACE\n\n\nSUPPOSING that Truth is a woman--what then? Is there not gro'

## 3 char model

### Create inputs

Create a list of every 4th character, starting at the 0th, 1st, 2nd, then 3rd characters

In [10]:
cs=3
c1_dat = [idx[i] for i in range(0, len(idx)-1-cs, cs)]
c2_dat = [idx[i+1] for i in range(0, len(idx)-1-cs, cs)]
c3_dat = [idx[i+2] for i in range(0, len(idx)-1-cs, cs)]
c4_dat = [idx[i+3] for i in range(0, len(idx)-1-cs, cs)]

In [13]:
print(c1_dat[:12])
print(c2_dat[:12])
print(c3_dat[:12])
print(c4_dat[:12])

[40, 30, 29, 1, 40, 43, 31, 61, 2, 74, 2, 2]
[42, 25, 1, 43, 40, 33, 2, 54, 44, 73, 62, 54]
[29, 27, 1, 45, 39, 38, 73, 73, 71, 61, 72, 2]
[30, 29, 1, 40, 43, 31, 61, 2, 74, 2, 2, 76]


Our inputs

In [14]:
x1 = np.stack(c1_dat[:-2])
x2 = np.stack(c2_dat[:-2])
x3 = np.stack(c3_dat[:-2])

In [17]:
print(x1[:10])

[40 30 29  1 40 43 31 61  2 74]


Our output

In [18]:
y = np.stack(c4_dat[:-2])

The first 4 inputs and outputs

In [19]:
x1[:4], x2[:4], x3[:4]

(array([40, 30, 29,  1]), array([42, 25,  1, 43]), array([29, 27,  1, 45]))

In [20]:
y[:4]

array([30, 29,  1, 40])

In [21]:
x1.shape, y.shape

((200295,), (200295,))

The number of latent factors to create (i.e. the size of the embedding matrix)

In [22]:
n_fac = 42

Create inputs and embedding outputs for each of our 3 character inputs

In [23]:
def embedding_input(name, n_in, n_out):
    inp = Input(shape=(1,), dtype='int64', name=name)
    emb = Embedding(n_in, n_out, input_length=1)(inp)
    return inp, Flatten()(emb)

In [24]:
c1_in, c1 = embedding_input('c1', vocab_size, n_fac)
c2_in, c2 = embedding_input('c2', vocab_size, n_fac)
c3_in, c3 = embedding_input('c3', vocab_size, n_fac)

In [26]:
print(c1_in)
print(c1)

Tensor("c1:0", shape=(?, 1), dtype=int64)
Tensor("flatten_1/Reshape:0", shape=(?, ?), dtype=float32)


### Create and train model

Pick a size for our hidden state

In [27]:
n_hidden = 256

This is the 'green arrow' from our diagram - the layer operation from input to hidden.

In [28]:
dense_in = Dense(n_hidden, activation='relu')

Our first hidden activation is simply this function applied to the result of the embedding of the first character.

In [29]:
c1_hidden = dense_in(c1)

This is the 'orange arrow' from our diagram - the layer operation from hidden to hidden.

In [30]:
dense_hidden = Dense(n_hidden, activation='tanh')

Our second and third hidden activations sum up the previous hidden state (after applying dense_hidden) to the new input state.

In [31]:
c2_dense = dense_in(c2)
hidden_2 = dense_hidden(c1_hidden)
c2_hidden = add([c2_dense, hidden_2])

In [32]:
c3_dense = dense_in(c3)
hidden_3 = dense_hidden(c2_hidden)
c3_hidden = add([c3_dense, hidden_3])

This is the 'blue arrow' from our diagram - the layer operation from hidden to output.

In [33]:
dense_out = Dense(vocab_size, activation='softmax')

The third hidden state is the input to our output layer.

In [34]:
c4_out = dense_out(c3_hidden)

In [35]:
model = Model([c1_in, c2_in, c3_in], c4_out)

In [36]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam())

In [37]:
model.optimizer.lr=0.000001

In [38]:
model.fit([x1, x2, x3], y, batch_size=64, epochs=4)

Epoch 1/4
200295/200295 [==============================] - 18s - loss: 4.4143    
Epoch 2/4
200295/200295 [==============================] - 17s - loss: 4.3080    
Epoch 3/4
200295/200295 [==============================] - 17s - loss: 4.0688    
Epoch 4/4
200295/200295 [==============================] - 17s - loss: 3.6806    


In [39]:
model.optimizer.lr=0.01

In [40]:
model.fit([x1, x2, x3], y, batch_size=64, epochs=4)

Epoch 1/4
200295/200295 [==============================] - 17s - loss: 3.3399    
Epoch 2/4
200295/200295 [==============================] - 17s - loss: 3.1971    
Epoch 3/4
200295/200295 [==============================] - 17s - loss: 3.1442    
Epoch 4/4
200295/200295 [==============================] - 17s - loss: 3.1195    


In [41]:
model.optimizer.lr=0.000001

In [42]:
model.fit([x1, x2, x3], y, batch_size=64, epochs=4)

Epoch 1/4
200295/200295 [==============================] - 17s - loss: 3.1044    
Epoch 2/4
200295/200295 [==============================] - 17s - loss: 3.0930    
Epoch 3/4
200295/200295 [==============================] - 17s - loss: 3.0837    
Epoch 4/4
200295/200295 [==============================] - 17s - loss: 3.0755    


In [43]:
model.optimizer.lr=0.01

In [44]:
model.fit([x1, x2, x3], y, batch_size=64, epochs=4)

Epoch 1/4
200295/200295 [==============================] - 17s - loss: 3.0678    
Epoch 2/4
200295/200295 [==============================] - 17s - loss: 3.0605    
Epoch 3/4
200295/200295 [==============================] - 17s - loss: 3.0532    
Epoch 4/4
200295/200295 [==============================] - 17s - loss: 3.0459    


### Test model

In [54]:
def get_next(inp):
    idxs = [char_indices[c] for c in inp]
    arrs = [np.array(i)[np.newaxis] for i in idxs]
    p = model.predict(arrs)
    print(p)
    i = np.argmax(p)
    print(">{}<".format(chars[2]))
    return chars[i]

In [56]:
print(chars)

['\x00', '\n', ' ', '!', '"', "'", '(', ')', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '=', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'Æ', 'ä', 'æ', 'é', 'ë']


In [55]:
get_next('phi')

[[  1.0256e-04   1.8753e-02   1.3744e-01   1.1306e-03   3.1811e-03   7.9774e-04   9.9588e-04
    7.2144e-04   1.4071e-02   6.4345e-03   4.3669e-03   4.1981e-04   7.6250e-04   6.0239e-04
    6.4321e-04   4.8032e-04   4.5058e-04   4.8008e-04   5.8927e-04   5.4135e-04   2.9757e-04
    1.5726e-03   1.1654e-03   5.8807e-04   7.7937e-04   1.8025e-03   1.0368e-03   9.9284e-04
    9.0306e-04   2.1545e-03   9.3952e-04   9.1229e-04   1.3410e-03   2.3757e-03   4.9868e-04
    4.5663e-04   9.5125e-04   1.1771e-03   1.2183e-03   1.5232e-03   1.0937e-03   2.7541e-04
    1.3664e-03   1.8447e-03   2.7689e-03   9.3024e-04   6.2150e-04   1.0845e-03   2.7143e-04
    6.4946e-04   2.2975e-04   3.8774e-04   4.9055e-04   5.8435e-04   5.3632e-02   1.0054e-02
    2.6022e-02   3.1555e-02   7.7869e-02   2.1587e-02   1.6923e-02   3.5692e-02   5.8725e-02
    1.1932e-03   4.0349e-03   3.4891e-02   2.1731e-02   6.2065e-02   5.3784e-02   1.5693e-02
    1.3354e-03   4.0081e-02   6.1480e-02   7.4143e-02   2.4180e-02   9

' '

In [47]:
get_next(' th')

' '

In [48]:
get_next(' an')

' '

## Our first RNN!

### Create inputs

This is the size of our unrolled RNN.

In [57]:
cs=8

For each of 0 through 7, create a list of every 8th character with that starting point. These will be the 8 inputs to out model.

In [58]:
c_in_dat = [[idx[i+n] for i in range(0, len(idx)-1-cs, cs)]
            for n in range(cs)]

Then create a list of the next character in each of these series. This will be the labels for our model.

In [59]:
c_out_dat = [idx[i+cs] for i in range(0, len(idx)-1-cs, cs)]

In [60]:
xs = [np.stack(c[:-2]) for c in c_in_dat]

In [61]:
len(xs), xs[0].shape

(8, (75109,))

In [62]:
y = np.stack(c_out_dat[:-2])

So each column below is one series of 8 characters from the text.

In [63]:
[xs[n][:cs] for n in range(cs)]

[array([40,  1, 33,  2, 72, 67, 73,  2]),
 array([42,  1, 38, 44,  2,  9, 61, 73]),
 array([29, 43, 31, 71, 54,  9, 58, 61]),
 array([30, 45,  2, 74,  2, 76, 67, 58]),
 array([25, 40, 73, 73, 76, 61, 24, 71]),
 array([27, 40, 61, 61, 68, 54,  2, 58]),
 array([29, 39, 54,  2, 66, 73, 33,  2]),
 array([ 1, 43, 73, 62, 54,  2, 72, 67])]

...and this is the next character after each sequence.

In [64]:
y[:cs]

array([ 1, 33,  2, 72, 67, 73,  2, 68])

In [65]:
n_fac = 42

### Create and train model

In [66]:
def embedding_input(name, n_in, n_out):
    inp = Input(shape=(1,), dtype='int64', name=name+'_in')
    emb = Embedding(n_in, n_out, input_length=1, name=name+'_emb')(inp)
    return inp, Flatten()(emb)

In [67]:
c_ins = [embedding_input('c'+str(n), vocab_size, n_fac) for n in range(cs)]

In [68]:
n_hidden = 256

In [69]:
dense_in = Dense(n_hidden, activation='relu')
dense_hidden = Dense(n_hidden, activation='relu', kernel_initializer='identity')
dense_out = Dense(vocab_size, activation='softmax')

The first character of each sequence goes through dense_in(), to create our first hidden activations.

In [70]:
hidden = dense_in(c_ins[0][1])  # [0] picks up the first char, [1] picks up the emb (see embedding_input)

Then for each successive layer we combine the output of dense_in() on the next character with the output of dense_hidden() on the current hidden state, to create the new hidden state.

In [71]:
for i in range(1,cs):
    c_dense = dense_in(c_ins[i][1])
    hidden = dense_hidden(hidden)
    hidden = add([c_dense, hidden])

Putting the final hidden state through dense_out() gives us our output.

In [72]:
c_out = dense_out(hidden)

So now we can create our model.

In [73]:
model = Model([c[0] for c in c_ins], c_out)
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam())

In [74]:
model.fit(xs, y, batch_size=64, epochs=12)

Epoch 1/12
75109/75109 [==============================] - 11s - loss: 2.5347    
Epoch 2/12
75109/75109 [==============================] - 11s - loss: 2.2573    
Epoch 3/12
75109/75109 [==============================] - 11s - loss: 2.1479    
Epoch 4/12
75109/75109 [==============================] - 11s - loss: 2.0792    
Epoch 5/12
75109/75109 [==============================] - 11s - loss: 2.0250    
Epoch 6/12
75109/75109 [==============================] - 11s - loss: 1.9784    
Epoch 7/12
75109/75109 [==============================] - 11s - loss: 1.9392    
Epoch 8/12
75109/75109 [==============================] - 11s - loss: 1.9039    
Epoch 9/12
75109/75109 [==============================] - 11s - loss: 1.8745    
Epoch 10/12
75109/75109 [==============================] - 11s - loss: 1.8459    
Epoch 11/12
75109/75109 [==============================] - 11s - loss: 1.8189    
Epoch 12/12
75109/75109 [==============================] - 11s - loss: 1.7940    


In [79]:
model.fit(xs, y, batch_size=64, epochs=12)

Epoch 1/12
75109/75109 [==============================] - 11s - loss: 1.7718    
Epoch 2/12
75109/75109 [==============================] - 11s - loss: 1.7534    
Epoch 3/12
75109/75109 [==============================] - 11s - loss: 1.7325    
Epoch 4/12
75109/75109 [==============================] - 11s - loss: 1.7151    
Epoch 5/12
75109/75109 [==============================] - 11s - loss: 1.6995    
Epoch 6/12
75109/75109 [==============================] - 11s - loss: 1.6858    
Epoch 7/12
75109/75109 [==============================] - 11s - loss: 1.6695    
Epoch 8/12
75109/75109 [==============================] - 11s - loss: 1.6574    
Epoch 9/12
75109/75109 [==============================] - 11s - loss: 1.6458    
Epoch 10/12
75109/75109 [==============================] - 11s - loss: 1.6311    
Epoch 11/12
75109/75109 [==============================] - 11s - loss: 1.6215    
Epoch 12/12
75109/75109 [==============================] - 11s - loss: 1.6120    


### Test model

In [75]:
def get_next(inp):
    idxs = [np.array(char_indices[c])[np.newaxis] for c in inp]
    p = model.predict(idxs)
    return chars[np.argmax(p)]

In [80]:
get_next('for thos')

'e'

In [81]:
get_next('part of ')

't'

In [82]:
get_next('queens a')

'n'

## Our first RNN with keras!

In [108]:
n_hidden, n_fac, cs, vocab_size = (256, 42, 8, 86)

This is nearly exactly equivalent to the RNN we built ourselves in the previous section.

In [109]:
model=Sequential([
        Embedding(vocab_size, n_fac, input_length=cs),
        SimpleRNN(n_hidden, activation='relu', recurrent_initializer='identity'),
        Dense(vocab_size, activation='softmax')
    ])

In [110]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 8, 42)             3612      
_________________________________________________________________
simple_rnn_3 (SimpleRNN)     (None, 256)               76544     
_________________________________________________________________
dense_9 (Dense)              (None, 86)                22102     
Total params: 102,258
Trainable params: 102,258
Non-trainable params: 0
_________________________________________________________________


In [111]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam())

In [112]:
#model.fit(np.stack(xs,1), y, batch_size=64, epochs=8)    # doesn't work (it seems it doesn't like the extra 1 dim)
model.fit(np.stack(xs,1).squeeze(), y, batch_size=64, epochs=8)

Epoch 1/8
75109/75109 [==============================] - 7s - loss: 2.8113     
Epoch 2/8
75109/75109 [==============================] - 7s - loss: 2.2860     
Epoch 3/8
75109/75109 [==============================] - 7s - loss: 2.0873     
Epoch 4/8
75109/75109 [==============================] - 7s - loss: 1.9499     
Epoch 5/8
75109/75109 [==============================] - 7s - loss: 1.8461     
Epoch 6/8
75109/75109 [==============================] - 7s - loss: 1.7639     
Epoch 7/8
75109/75109 [==============================] - 7s - loss: 1.6981     
Epoch 8/8
75109/75109 [==============================] - 7s - loss: 1.6428     


In [113]:
def get_next_keras(inp):
    idxs = [char_indices[c] for c in inp]
    arrs = np.array(idxs)[np.newaxis,:]
    p = model.predict(arrs)[0]
    return chars[np.argmax(p)]

In [114]:
get_next_keras('this is ')

't'

In [115]:
get_next_keras('part of ')

't'

In [116]:
get_next_keras('e libert')

' '

In [96]:
text="tnaursiteaurtct887"
text[-7:]

'rtct887'

In [101]:
text = "The life is"
for i in range(0,50):
    text+=get_next_keras(text[-8:])
print(text)

The life is the moral the senses of the senses of the senses 


## Returning sequences

### Create inputs

To use a sequence model, we can leave our input unchanged - but we have to change our output to a sequence (of course!)

Here, c_out_dat is identical to c_in_dat, but moved across 1 character.

In [ ]:
#c_in_dat = [[idx[i+n] for i in range(0, len(idx)-1-cs, cs)]
#            for n in range(cs)]
c_out_dat = [[idx[i+n] for i in range(1, len(idx)-cs, cs)]
            for n in range(cs)]

In [ ]:
xs = [np.stack(c[:-2]) for c in c_in_dat]  # repeated here to restore the original after the previous np.stack.squeeze
ys = [np.stack(c[:-2]) for c in c_out_dat]

Reading down each column shows one set of inputs and outputs.

In [ ]:
[xs[n][:cs] for n in range(cs)]

In [ ]:
[ys[n][:cs] for n in range(cs)]

### Create and train model

In [ ]:
dense_in = Dense(n_hidden, activation='relu')
dense_hidden = Dense(n_hidden, activation='relu', kernel_initializer='identity')
dense_out = Dense(vocab_size, activation='softmax', name='output')

We're going to pass a vector of all zeros as our starting point - here's our input layers for that:

In [ ]:
inp1 = Input(shape=(n_fac,), name='zeros')
hidden = dense_in(inp1)

In [ ]:
outs = []

for i in range(cs):
    c_dense = dense_in(c_ins[i][1])
    hidden = dense_hidden(hidden)
    hidden = add([c_dense, hidden])
    # every layer now has an output
    outs.append(dense_out(hidden))

In [ ]:
model = Model([inp1] + [c[0] for c in c_ins], outs)
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam())

In [ ]:
zeros = np.tile(np.zeros(n_fac), (len(xs[0]),1))
zeros.shape

In [ ]:
model.fit([zeros]+xs, ys, batch_size=64, epochs=12)

### Test model

In [ ]:
def get_nexts(inp):
    idxs = [char_indices[c] for c in inp]
    arrs = [np.array(i)[np.newaxis] for i in idxs]
    p = model.predict([np.zeros(n_fac)[np.newaxis,:]] + arrs)
    print(list(inp))
    return [chars[np.argmax(o)] for o in p]

In [ ]:
get_nexts(' this is')

In [ ]:
get_nexts(' part of')

### Sequence model with keras

In [ ]:
n_hidden, n_fac, cs, vocab_size

To convert our previous keras model into a sequence model, simply add the 'return_sequences=True' parameter, and add TimeDistributed() around our dense layer.

In [ ]:
model=Sequential([
        Embedding(vocab_size, n_fac, input_length=cs),
        SimpleRNN(n_hidden, return_sequences=True, activation='relu', recurrent_initializer='identity'),
        TimeDistributed(Dense(vocab_size, activation='softmax')),
    ])

In [ ]:
model.summary()

In [ ]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam())

In [ ]:
x_rnn=np.stack(xs, axis=1)
# y_rnn=np.expand_dims(np.stack(ys, axis=1), -1)  # doesn't work (it seems it doesn't like the extra 1 dim)
y_rnn=np.stack(ys, axis=1)

In [ ]:
x_rnn.shape, y_rnn.shape

In [ ]:
model.fit(x_rnn, y_rnn, batch_size=64, epochs=8)

In [ ]:
def get_nexts_keras(inp):
    idxs = [char_indices[c] for c in inp]
    arr = np.array(idxs)[np.newaxis,:]
    p = model.predict(arr)[0]
    print(list(inp))
    return [chars[np.argmax(o)] for o in p]

In [ ]:
get_nexts_keras(' this is')

### One-hot sequence model with keras

This is the keras version of the theano model that we're about to create.

In [ ]:
model=Sequential([
        SimpleRNN(n_hidden, return_sequences=True, input_shape=(cs, vocab_size),
                  activation='relu', recurrent_initializer='identity'),
        TimeDistributed(Dense(vocab_size, activation='softmax')),
    ])
model.compile(loss='categorical_crossentropy', optimizer=Adam())

In [ ]:
oh_ys = [to_categorical(o, vocab_size) for o in ys]
oh_y_rnn=np.stack(oh_ys, axis=1)

oh_xs = [to_categorical(o, vocab_size) for o in xs]
oh_x_rnn=np.stack(oh_xs, axis=1)

oh_x_rnn.shape, oh_y_rnn.shape

In [ ]:
model.fit(oh_x_rnn, oh_y_rnn, batch_size=64, epochs=8)

In [ ]:
def get_nexts_oh(inp):
    idxs = np.array([char_indices[c] for c in inp])
    arr = to_categorical(idxs, vocab_size)
    p = model.predict(arr[np.newaxis,:])[0]
    print(list(inp))
    return [chars[np.argmax(o)] for o in p]

In [ ]:
get_nexts_oh(' this is')

## Stateful model with keras

In [ ]:
bs=64

A stateful model is easy to create (just add "stateful=True") but harder to train. We had to add batchnorm and use LSTM to get reasonable results.

When using stateful in keras, you have to also add 'batch_input_shape' to the first layer, and fix the batch size there.

In [ ]:
model=Sequential([
        Embedding(vocab_size, n_fac, input_length=cs, batch_input_shape=(bs,8)),
        BatchNormalization(),
        LSTM(n_hidden, return_sequences=True, stateful=True),
        TimeDistributed(Dense(vocab_size, activation='softmax')),
    ])

In [ ]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam())

Since we're using a fixed batch shape, we have to ensure our inputs and outputs are a even multiple of the batch size.

In [ ]:
mx = len(x_rnn)//bs*bs

In [ ]:
model.fit(x_rnn[:mx], y_rnn[:mx], batch_size=bs, epochs=4, shuffle=False)

In [ ]:
model.optimizer.lr=1e-4

In [ ]:
model.fit(x_rnn[:mx], y_rnn[:mx], batch_size=bs, epochs=4, shuffle=False)

In [ ]:
model.fit(x_rnn[:mx], y_rnn[:mx], batch_size=bs, epochs=4, shuffle=False)

## Theano RNN

In [ ]:
n_input = vocab_size
n_output = vocab_size

Using raw theano, we have to create our weight matrices and bias vectors ourselves - here are the functions we'll use to do so (using glorot initialization).

The return values are wrapped in `shared()`, which is how we tell theano that it can manage this data (copying it to and from the GPU as necessary).

In [ ]:
def init_wgts(rows, cols): 
    scale = math.sqrt(2/rows)
    return shared(normal(scale=scale, size=(rows, cols)).astype(np.float32))
def init_bias(rows): 
    return shared(np.zeros(rows, dtype=np.float32))

We return the weights and biases together as a tuple. For the hidden weights, we'll use an identity initialization (as recommended by [Hinton](https://arxiv.org/abs/1504.00941).)

In [ ]:
def wgts_and_bias(n_in, n_out): 
    return init_wgts(n_in, n_out), init_bias(n_out)
def id_and_bias(n): 
    return shared(np.eye(n, dtype=np.float32)), init_bias(n)

Theano doesn't actually do any computations until we explicitly compile and evaluate the function (at which point it'll be turned into CUDA code and sent off to the GPU). So our job is to describe the computations that we'll want theano to do - the first step is to tell theano what inputs we'll be providing to our computation:

In [ ]:
t_inp = T.matrix('inp')
t_outp = T.matrix('outp')
t_h0 = T.vector('h0')
lr = T.scalar('lr')

all_args = [t_h0, t_inp, t_outp, lr]

Now we're ready to create our intial weight matrices.

In [ ]:
W_h = id_and_bias(n_hidden)
W_x = wgts_and_bias(n_input, n_hidden)
W_y = wgts_and_bias(n_hidden, n_output)
w_all = list(chain.from_iterable([W_h, W_x, W_y]))

Theano handles looping by using the [GPU scan](http://http.developer.nvidia.com/GPUGems3/gpugems3_ch39.html) operation. We have to tell theano what to do at each step through the scan - this is the function we'll use, which does a single forward pass for one character:

In [ ]:
def step(x, h, W_h, b_h, W_x, b_x, W_y, b_y):
    # Calculate the hidden activations
    h = nnet.relu(T.dot(x, W_x) + b_x + T.dot(h, W_h) + b_h)
    # Calculate the output activations
    y = nnet.softmax(T.dot(h, W_y) + b_y)
    # Return both (the 'Flatten()' is to work around a theano bug)
    return h, T.flatten(y, 1)

Now we can provide everything necessary for the scan operation, so we can setup that up - we have to pass in the function to call at each step, the sequence to step through, the initial values of the outputs, and any other arguments to pass to the step function.

In [ ]:
[v_h, v_y], _ = theano.scan(step, sequences=t_inp, 
                            outputs_info=[t_h0, None], non_sequences=w_all)

We can now calculate our loss function, and *all* of our gradients, with just a couple of lines of code!

In [ ]:
error = nnet.categorical_crossentropy(v_y, t_outp).sum()
g_all = T.grad(error, w_all)

We even have to show theano how to do SGD - so we set up this dictionary of updates to complete after every forward pass, which apply to standard SGD update rule to every weight.

In [ ]:
def upd_dict(wgts, grads, lr): 
    return OrderedDict({w: w-g*lr for (w,g) in zip(wgts,grads)})

upd = upd_dict(w_all, g_all, lr)

We're finally ready to compile the function!

In [ ]:
fn = theano.function(all_args, error, updates=upd, allow_input_downcast=True)

In [ ]:
X = oh_x_rnn
Y = oh_y_rnn
X.shape, Y.shape

To use it, we simply loop through our input data, calling the function compiled above, and printing our progress from time to time.

In [ ]:
err=0.0; l_rate=0.01
for i in range(len(X)): 
    err+=fn(np.zeros(n_hidden), X[i], Y[i], l_rate)
    if i % 1000 == 999: 
        print ("Error:{:.3f}".format(err/1000))
        err=0.0

In [ ]:
f_y = theano.function([t_h0, t_inp], v_y, allow_input_downcast=True)

In [ ]:
pred = np.argmax(f_y(np.zeros(n_hidden), X[6]), axis=1)

In [ ]:
act = np.argmax(X[6], axis=1)

In [ ]:
[indices_char[o] for o in act]

In [ ]:
[indices_char[o] for o in pred]

## Pure python RNN!

### Set up basic functions

Now we're going to try to repeat the above theano RNN, using just pure python (and numpy). Which means, we have to do everything ourselves, including defining the basic functions of a neural net! Below are all of the definitions, along with tests to check that they give the same answers as theano. The functions ending in `_d` are the derivatives of each function.

In [ ]:
def sigmoid(x): return 1/(1+np.exp(-x))
def sigmoid_d(x): 
    output = sigmoid(x)
    return output*(1-output)

In [ ]:
def relu(x): return np.maximum(0., x)
def relu_d(x): return (x > 0.)*1.

In [ ]:
relu(np.array([3.,-3.])), relu_d(np.array([3.,-3.]))

In [ ]:
def dist(a,b): return pow(a-b,2)
def dist_d(a,b): return 2*(a-b)

In [ ]:
import pdb

In [ ]:
eps = 1e-7
def x_entropy(pred, actual): 
    return -np.sum(actual * np.log(np.clip(pred, eps, 1-eps)))
def x_entropy_d(pred, actual): return -actual/pred

In [ ]:
def softmax(x): return np.exp(x)/np.exp(x).sum()

In [ ]:
def softmax_d(x):
    sm = softmax(x)
    res = np.expand_dims(-sm,-1)*sm
    res[np.diag_indices_from(res)] = sm*(1-sm)
    return res

In [ ]:
test_preds = np.array([0.2,0.7,0.1])
test_actuals = np.array([0.,1.,0.])
nnet.categorical_crossentropy(test_preds, test_actuals).eval()

In [ ]:
x_entropy(test_preds, test_actuals)

In [ ]:
test_inp = T.dvector()
test_out = nnet.categorical_crossentropy(test_inp, test_actuals)
test_grad = theano.function([test_inp], T.grad(test_out, test_inp))

In [ ]:
test_grad(test_preds)

In [ ]:
x_entropy_d(test_preds, test_actuals)

In [ ]:
pre_pred = random(oh_x_rnn[0][0].shape)
preds = softmax(pre_pred)
actual = oh_x_rnn[0][0]

In [ ]:
loss=x_entropy
loss_d=x_entropy_d

In [ ]:
np.allclose(softmax_d(pre_pred).dot(loss_d(preds,actual)), preds-actual)

In [ ]:
softmax(test_preds)

In [ ]:
nnet.softmax(test_preds).eval()

In [ ]:
test_out = T.flatten(nnet.softmax(test_inp))

In [ ]:
test_grad = theano.function([test_inp], theano.gradient.jacobian(test_out, test_inp))

In [ ]:
test_grad(test_preds)

In [ ]:
softmax_d(test_preds)

In [ ]:
act=relu
act_d = relu_d

We also have to define our own scan function. Since we're not worrying about running things in parallel, it's very simple to implement:

In [ ]:
def scan(fn, start, seq):
    res = []
    prev = start
    for s in seq:
        app = fn(prev, s)
        res.append(app)
        prev = app
    return res

...for instance, `scan` on `+` is the cumulative sum.

In [ ]:
scan(lambda prev,curr: prev+curr, 0, range(5))

### Set up training

Let's now build the functions to do the forward and backward passes of our RNN. First, define our data and shape.

In [ ]:
inp = oh_x_rnn
outp = oh_y_rnn
n_input = vocab_size
n_output = vocab_size

In [ ]:
inp.shape, outp.shape

Here's the function to do a single forward pass of an RNN, for a single character.

In [ ]:
def one_char(prev, item):
    # Previous state
    tot_loss, pre_hidden, pre_pred, hidden, ypred = prev
    # Current inputs and output
    x, y = item
    pre_hidden = np.dot(x,w_x) + np.dot(hidden,w_h)
    hidden = act(pre_hidden)
    pre_pred = np.dot(hidden,w_y)
    ypred = softmax(pre_pred)
    return (
        # Keep track of loss so we can report it
        tot_loss+loss(ypred, y),
        # Used in backprop
        pre_hidden, pre_pred, 
        # Used in next iteration
        hidden, 
        # To provide predictions
        ypred)

We use `scan` to apply the above to a whole sequence of characters.

In [ ]:
def get_chars(n): return zip(inp[n], outp[n])
def one_fwd(n): return scan(one_char, (0,0,0,np.zeros(n_hidden),0), get_chars(n))

Now we can define the backward step. We use a loop to go through every element of the sequence. The derivatives are applying the chain rule to each step, and accumulating the gradients across the sequence.

In [ ]:
# "Columnify" a vector
def col(x): return x[:,newaxis]

def one_bkwd(args, n):
    global w_x,w_y,w_h

    i=inp[n]  # 8x86
    o=outp[n] # 8x86
    d_pre_hidden = np.zeros(n_hidden) # 256
    for p in reversed(range(len(i))):
        totloss, pre_hidden, pre_pred, hidden, ypred = args[p]
        x=i[p] # 86
        y=o[p] # 86
        d_pre_pred = softmax_d(pre_pred).dot(loss_d(ypred,y))  # 86
        d_pre_hidden = (np.dot(d_pre_hidden, w_h.T) 
                        + np.dot(d_pre_pred,w_y.T)) * act_d(pre_hidden) # 256

        # d(loss)/d(w_y) = d(loss)/d(pre_pred) * d(pre_pred)/d(w_y)
        w_y -= col(hidden) * d_pre_pred * alpha
        # d(loss)/d(w_h) = d(loss)/d(pre_hidden[p-1]) * d(pre_hidden[p-1])/d(w_h)
        if (p>0): w_h -= args[p-1][3].dot(d_pre_hidden) * alpha
        w_x -= col(x)*d_pre_hidden * alpha
    return d_pre_hidden

Now we can set up our initial weight matrices. Note that we're not using bias at all in this example, in order to keep things simpler.

In [ ]:
scale=math.sqrt(2./n_input)
w_x = normal(scale=scale, size=(n_input,n_hidden))
w_y = normal(scale=scale, size=(n_hidden, n_output))
w_h = np.eye(n_hidden, dtype=np.float32)

Our loop looks much like the theano loop in the previous section, except that we have to call the backwards step ourselves.

In [ ]:
overallError=0
alpha=0.0001
for n in range(10000):
    res = one_fwd(n)
    overallError+=res[-1][0]
    deriv = one_bkwd(res, n)
    if(n % 1000 == 999):
        print ("Error:{:.4f}; Gradient:{:.5f}".format(
                overallError/1000, np.linalg.norm(deriv)))
        overallError=0

## Keras GRU

Identical to the last keras rnn, but a GRU!

In [ ]:
# No inner_init='identity' in the model, 
# "Identity matrix initializer can only be used for 2D square matrices" (!?)
model=Sequential([
        GRU(n_hidden, return_sequences=True, input_shape=(cs, vocab_size),
                  activation='relu'),
        TimeDistributed(Dense(vocab_size, activation='softmax')),
    ])
model.compile(loss='categorical_crossentropy', optimizer=Adam())

In [ ]:
model.fit(oh_x_rnn, oh_y_rnn, batch_size=64, epochs=8)

In [ ]:
get_nexts_oh(' this is')

## Theano GRU

### Separate weights

The theano GRU looks just like the simple theano RNN, except for the use of the reset and update gates. Each of these gates requires its own hidden and input weights, so we add those to our weight matrices.

In [ ]:
W_h = id_and_bias(n_hidden)
W_x = init_wgts(n_input, n_hidden)
W_y = wgts_and_bias(n_hidden, n_output)
rW_h = init_wgts(n_hidden, n_hidden)
rW_x = wgts_and_bias(n_input, n_hidden)
uW_h = init_wgts(n_hidden, n_hidden)
uW_x = wgts_and_bias(n_input, n_hidden)
w_all = list(chain.from_iterable([W_h, W_y, uW_x, rW_x]))
w_all.extend([W_x, uW_h, rW_h])

Here's the definition of a gate - it's just a sigmoid applied to the addition of the dot products of the input vectors.

In [ ]:
def gate(x, h, W_h, W_x, b_x):
    return nnet.sigmoid(T.dot(x, W_x) + b_x + T.dot(h, W_h))

Our step is nearly identical to before, except that we multiply our hidden state by our reset gate, and we update our hidden state based on the update gate.

In [ ]:
def step(x, h, W_h, b_h, W_y, b_y, uW_x, ub_x, rW_x, rb_x, W_x, uW_h, rW_h):
    reset = gate(x, h, rW_h, rW_x, rb_x)
    update = gate(x, h, uW_h, uW_x, ub_x)
    h_new = gate(x, h * reset, W_h, W_x, b_h)
    h = update*h + (1-update)*h_new
    y = nnet.softmax(T.dot(h, W_y) + b_y)
    return h, T.flatten(y, 1)

Everything from here on is identical to our simple RNN in theano.

In [ ]:
[v_h, v_y], _ = theano.scan(step, sequences=t_inp, 
                            outputs_info=[t_h0, None], non_sequences=w_all)

In [ ]:
error = nnet.categorical_crossentropy(v_y, t_outp).sum()
g_all = T.grad(error, w_all)

In [ ]:
upd = upd_dict(w_all, g_all, lr)
fn = theano.function(all_args, error, updates=upd, allow_input_downcast=True)

In [ ]:
err=0.0; l_rate=0.1
for i in range(len(X)): 
    err+=fn(np.zeros(n_hidden), X[i], Y[i], l_rate)
    if i % 1000 == 999: 
        l_rate *= 0.95
        print ("Error:{:.2f}".format(err/1000))
        err=0.0

### Combined weights

We can make the previous section simpler and faster by concatenating the hidden and input matrices and inputs together. We're not going to step through this cell by cell - you'll see it's identical to the previous section except for this concatenation.

In [ ]:
W = (shared(np.concatenate([np.eye(n_hidden), normal(size=(n_input, n_hidden))])
            .astype(np.float32)), init_bias(n_hidden))

rW = wgts_and_bias(n_input+n_hidden, n_hidden)
uW = wgts_and_bias(n_input+n_hidden, n_hidden)
W_y = wgts_and_bias(n_hidden, n_output)
w_all = list(chain.from_iterable([W, W_y, uW, rW]))

In [ ]:
def gate(m, W, b): return nnet.sigmoid(T.dot(m, W) + b)

In [ ]:
def step(x, h, W, b, W_y, b_y, uW, ub, rW, rb):
    m = T.concatenate([h, x])
    reset = gate(m, rW, rb)
    update = gate(m, uW, ub)
    m = T.concatenate([h*reset, x])
    h_new = gate(m, W, b)
    h = update*h + (1-update)*h_new
    y = nnet.softmax(T.dot(h, W_y) + b_y)
    return h, T.flatten(y, 1)

In [ ]:
[v_h, v_y], _ = theano.scan(step, sequences=t_inp, 
                            outputs_info=[t_h0, None], non_sequences=w_all)

In [ ]:
def upd_dict(wgts, grads, lr): 
    return OrderedDict({w: w-g*lr for (w,g) in zip(wgts,grads)})

In [ ]:
error = nnet.categorical_crossentropy(v_y, t_outp).sum()
g_all = T.grad(error, w_all)

In [ ]:
upd = upd_dict(w_all, g_all, lr)
fn = theano.function(all_args, error, updates=upd, allow_input_downcast=True)

In [ ]:
err=0.0; l_rate=0.01
for i in range(len(X)): 
    err+=fn(np.zeros(n_hidden), X[i], Y[i], l_rate)
    if i % 1000 == 999: 
        print ("Error:{:.2f}".format(err/1000))
        err=0.0

### End